In [ ]:
import tkinter as tk
from tkinter import ttk, Menu, filedialog
from matplotlib.figure import Figure
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
import cv2
import os
from PIL import Image, ImageTk
from collections import defaultdict
import numpy as np
from ultralytics import YOLO

class VideoPlayer(tk.Tk):
    def __init__(self, video_path):
        super().__init__()
        self.title("Video Player GUI")
        self.geometry("1000x700")
        self.model = YOLO("med_yolo11n.pt")
        self.paused = False
        self.zoomed_frame = None
        self.prev_checkbox_status = {}
        self.track_history = defaultdict(list)
        self.cap = cv2.VideoCapture(video_path)

        self.create_menu()
        self.create_main_frames()
        self.create_left_panels()
        self.create_right_panels()
        self.create_bar_graph()
        self.create_controls()
        self.create_speed_slider()
        self.update_video()

    def create_menu(self):
        menu_bar = Menu(self)
        self.config(menu=menu_bar)
        file_menu = Menu(menu_bar, tearoff=0)
        file_menu.add_command(label="Open", command=self.open_file)
        file_menu.add_command(label="Save", command=self.save_file)
        file_menu.add_separator()
        file_menu.add_command(label="Exit", command=self.quit)
        menu_bar.add_cascade(label="File", menu=file_menu)
        edit_menu = Menu(menu_bar, tearoff=0)
        edit_menu.add_command(label="Undo", command=self.undo_action)
        edit_menu.add_command(label="Redo", command=self.redo_action)
        menu_bar.add_cascade(label="Edit", menu=edit_menu)
        help_menu = Menu(menu_bar, tearoff=0)
        help_menu.add_command(label="About", command=self.show_about)
        menu_bar.add_cascade(label="Help", menu=help_menu)

    def create_main_frames(self):
        self.left_frame = tk.Frame(self, width=300, height=700, padx=10, pady=10)
        self.left_frame.pack_propagate(False)
        self.left_frame.pack(side=tk.LEFT, fill=tk.Y, padx=5, pady=5)
        self.right_frame = tk.Frame(self, width=680, height=700, padx=10, pady=10)
        self.right_frame.pack_propagate(False)
        self.right_frame.pack(side=tk.RIGHT, fill=tk.BOTH, expand=True, padx=5, pady=5)

    def create_left_panels(self):
        self.top_left_panel = self.create_label_frame(self.left_frame, "Zoomed and Cropped Video")
        self.middle_left_panel = self.create_label_frame(self.left_frame, "Counted Types")
        self.bottom_left_panel = self.create_label_frame(self.left_frame, "Show Types")
        self.zoomed_label = tk.Label(self.top_left_panel)
        self.zoomed_label.pack(fill=tk.BOTH, expand=True)
        self.check_vars = [tk.IntVar() for _ in range(7)]
        self.clses = ["T1-Amorf_Bas", "T2-Akrozomal_Vakuol", "T3-Normal", "T4-Amorf_Bas-Kisa_Kuyruk", "T5-Amorf_Bas-Kisa_Kut_Kuyruk", "T6-Nuklear_Anomali", "T7-Uzun_Bas-Buyuk_Bas"]
        self.check_buttons = [self.create_check_button(self.bottom_left_panel, text, var) for text, var in zip(self.clses, self.check_vars)]

    def create_right_panels(self):
        self.video_panel = self.create_label_frame(self.right_frame, "Video", 680, 500)
        self.video_label = tk.Label(self.video_panel)
        self.video_label.pack(fill=tk.BOTH, expand=True)
        self.explanation_panel = self.create_label_frame(self.right_frame, "Explanation Area", 680, 100)
        self.explanation_label = tk.Label(self.explanation_panel, text="Explanation Area", wraplength=600)
        self.explanation_label.pack(fill=tk.X)

    def create_bar_graph(self):
        self.fig = Figure(figsize=(3, 2))
        self.ax = self.fig.add_subplot(111)
        self.canvas = FigureCanvasTkAgg(self.fig, master=self.middle_left_panel)
        self.canvas.draw()
        self.canvas.get_tk_widget().pack(fill=tk.BOTH, expand=True)
        self.update_bar_graph([5, 7, 3, 8, 4, 0, 0])

    def create_controls(self):
        self.play_button = tk.Button(self.right_frame, text="Play", command=self.play_video)
        self.play_button.pack(side=tk.LEFT, padx=5, pady=5)
        self.pause_button = tk.Button(self.right_frame, text="Pause", command=self.pause_video)
        self.pause_button.pack(side=tk.LEFT, padx=5, pady=5)
        self.save_button = tk.Button(self.right_frame, text="Save Frame", command=self.save_frame)
        self.save_button.pack(side=tk.LEFT, padx=5, pady=5)
        self.fullscreen_button = tk.Button(self.right_frame, text="Fullscreen", command=self.toggle_fullscreen)
        self.fullscreen_button.pack(side=tk.LEFT, padx=5, pady=5)

    def create_speed_slider(self):
        self.speed_slider = tk.Scale(self.right_frame, from_=1, to=100, orient=tk.HORIZONTAL, label="Speed")
        self.speed_slider.set(30)
        self.speed_slider.pack(side=tk.LEFT, padx=5, pady=5)

    def create_label_frame(self, parent, text, width=300, height=200):
        frame = tk.LabelFrame(parent, text=text, width=width, height=height)
        frame.pack_propagate(False)
        frame.pack(fill=tk.BOTH, expand=True, padx=5, pady=5)
        return frame

    def create_check_button(self, parent, text, var):
        chk = tk.Checkbutton(parent, text=text, variable=var, command=self.update_explanation_area)
        chk.pack(anchor=tk.W)
        return chk

    def open_file(self):
        filetypes = (("Video files", "*.mp4 *.avi *.mkv"), ("All files", "*.*"))
        filepath = filedialog.askopenfilename(title="Open Video File", initialdir=os.getcwd(), filetypes=filetypes)
        if filepath:
            self.change_video(filepath)

    def save_file(self):
        pass

    def undo_action(self):
        pass

    def redo_action(self):
        pass

    def show_about(self):
        pass

    def change_video(self, video_path):
        if self.cap.isOpened():
            self.cap.release()
        self.track_history.clear()
        self.cap = cv2.VideoCapture(video_path)

    def play_video(self):
        self.paused = False
        self.update_video()

    def pause_video(self):
        self.paused = True
        if hasattr(self, 'after_id'):
            self.after_cancel(self.after_id)

    def save_frame(self):
        ret, frame = self.cap.read()
        if ret:
            cv2.imwrite("saved_frame.png", frame)
            print("Frame saved as saved_frame.png")

    def toggle_fullscreen(self):
        self.attributes("-fullscreen", not self.attributes("-fullscreen"))

    def update_video(self):
        if self.paused:
            return
        ret, frame = self.cap.read()
        if ret:
            frame_with_rectangle = self.process_frame(frame)
            self.display_main_video(frame_with_rectangle)
            self.update_zoomed_frame(frame, self.detect_objects(frame)[1])  # Update the zoomed frame with the detected object
            self.update_periodic_tasks()
        self.after_id = self.after(self.speed_slider.get(), self.update_video)

    def process_frame(self, frame):
        frame_with_rectangle = frame.copy()
        x, y = 150, 150
        result, best_box = self.detect_objects(frame_with_rectangle)
        if result is not None:
            frame_with_rectangle = result
        self.draw_rectangle(frame_with_rectangle, best_box, x, y)
        return frame_with_rectangle

    def draw_rectangle(self, frame, best_box, x, y):
        rect_width, rect_height = 100, 100
        if best_box is not None:
            x, y, w, h = map(int, best_box)
        cv2.rectangle(frame, (x - rect_width // 2, y - rect_height // 2), (x + rect_width // 2, y + rect_height // 2), (0, 255, 0), 2)
        self.update_explanation(f"Rectangle Coordinates: ({x - rect_width // 2}, {y - rect_height // 2}), ({x + rect_width // 2}, {y + rect_height // 2})")

    def detect_objects(self, frame):
        clss = self.get_checked_indices() or [2]
        results = self.model.track(frame, imgsz=(640, 480), conf=0.4, iou=0.5, tracker="bytetrack.yaml", persist=True, classes=clss)
        annotated_frame, best_box = self.extract_best_box(results)
        if best_box is not None:
            self.update_zoomed_frame(frame, best_box)
        return annotated_frame, best_box

    def extract_best_box(self, results):
        best_box = None
        annotated_frame = None
        if results[0].boxes.id is not None:
            boxes, scores, classes = self.extract_boxes_scores_classes(results)
            best_box = self.find_most_probable_normal(boxes, scores, classes)
            track_ids = results[0].boxes.id.int().cpu().tolist()
            annotated_frame = self.annotate_frame(results, boxes, track_ids)
        return annotated_frame, best_box

    def extract_boxes_scores_classes(self, results):
        boxes = results[0].boxes.xywh.cpu().numpy()
        scores = results[0].boxes.conf.cpu().numpy()
        classes = results[0].boxes.cls.cpu().numpy()
        return boxes, scores, classes

    def find_most_probable_normal(self, boxes, scores, classes, normal_class_index=3):
        max_score = 0
        best_box = None
        for box, score, cls in zip(boxes, scores, classes):
            if cls == normal_class_index and score > max_score:
                max_score = score
                best_box = box
        return best_box

    def annotate_frame(self, results, boxes, track_ids):
        annotated_frame = results[0].plot(font_size=6)
        for box, track_id in zip(boxes, track_ids):
            x, y, w, h = box
            track = self.track_history[track_id]
            track.append((float(x), float(y)))
            if len(track) > 30:
                track.pop(0)
            points = np.hstack(track).astype(np.int32).reshape((-1, 1, 2))
            cv2.polylines(annotated_frame, [points], isClosed=False, color=(230, 230, 230), thickness=2)
        return annotated_frame

    def update_zoomed_frame(self, frame, best_box=None):
        height, width, _ = frame.shape
        cx, cy = self.get_zoom_center(best_box, width, height)
        x1, y1, x2, y2 = self.get_zoom_coordinates(cx, cy, width, height)
        
        # Debugging information
        print(f"Zoomed Frame Coordinates: ({x1}, {y1}), ({x2}, {y2})")
        
        self.zoomed_frame = frame[y1:y2, x1:x2].copy()
        
        # Check if the zoomed frame is empty
        if self.zoomed_frame.size == 0:
            print("Zoomed frame is empty!")
        else:
            print("Zoomed frame is not empty.")
        
        self.display_zoomed_video()  # Ensure the zoomed frame is displayed immediately


    def get_zoom_center(self, best_box, width, height):
        if best_box is not None:
            x, y, w, h = map(int, best_box)
            return x, y
        # Default center coordinates
        return width // 2, height // 2


    def get_zoom_coordinates(self, cx, cy, width, height, zoom_size=100):
        x1, y1 = max(0, cx - zoom_size // 2), max(0, cy - zoom_size // 2)
        x2, y2 = min(width, cx + zoom_size // 2), min(height, cy + zoom_size // 2)
        return x1, y1, x2, y2

    def display_zoomed_video(self):
        if self.zoomed_frame is not None:
            zoomed_frame_resized = cv2.resize(self.zoomed_frame, (self.zoomed_label.winfo_width(), self.zoomed_label.winfo_height()), interpolation=cv2.INTER_LINEAR)
            zoomed_frame_rgb = cv2.cvtColor(zoomed_frame_resized, cv2.COLOR_BGR2RGB)
            img = Image.fromarray(zoomed_frame_rgb)
            imgtk = ImageTk.PhotoImage(image=img)
            self.zoomed_label.imgtk = imgtk  # Keep a reference to avoid garbage collection
            self.zoomed_label.configure(image=imgtk)

    def update_video(self):
        if self.paused:
            return
        ret, frame = self.cap.read()
        if ret:
            frame_with_rectangle = self.process_frame(frame)
            self.display_main_video(frame_with_rectangle)
            _, best_box = self.detect_objects(frame)
            self.update_zoomed_frame(frame, best_box)  # Update the zoomed frame with the detected object
            self.update_periodic_tasks()
        self.after_id = self.after(self.speed_slider.get(), self.update_video)

    def display_main_video(self, frame):
        frame_resized = cv2.resize(frame, (self.video_label.winfo_width(), self.video_label.winfo_height()))
        frame_rgb = cv2.cvtColor(frame_resized, cv2.COLOR_BGR2RGB)
        img = Image.fromarray(frame_rgb)
        imgtk = ImageTk.PhotoImage(image=img)
        self.video_label.imgtk = imgtk  # Keep a reference to avoid garbage collection
        self.video_label.configure(image=imgtk)
    
    def update_explanation_area(self):
        current_checkbox_status = self.get_checkboxes_status()
        if current_checkbox_status != self.prev_checkbox_status:
            explanation_text = "Checkboxes Status:\n" + "\n".join([f"{k}: {'Checked' if v else 'Unchecked'}" for k, v in current_checkbox_status.items()])
            self.update_explanation(explanation_text)
            self.prev_checkbox_status = current_checkbox_status

    def update_periodic_tasks(self):
        if int(self.cap.get(cv2.CAP_PROP_POS_FRAMES)) % int(self.cap.get(cv2.CAP_PROP_FPS)) == 0:
            self.update_bar_graph([6, 8, 4, 9, 5, 0, 0])
        self.update_explanation_area()

    def update_bar_graph(self, data):
        labels = ['T1', 'T2', 'T3', 'T4', 'T5', 'T6', 'T7']
        self.ax.clear()
        self.ax.bar(range(len(data)), data)
        self.ax.set_xticks(range(len(data)))
        self.ax.set_xticklabels(labels)
        self.canvas.draw()

    def update_explanation(self, text):
        self.explanation_label.config(text=text)

    def get_checkboxes_status(self):
        return {chk.cget("text"): var.get() for chk, var in zip(self.check_buttons, self.check_vars)}

    def get_checked_indices(self):
        return [i for i, var in enumerate(self.check_vars) if var.get() == 1]


In [ ]:
import tkinter as tk
from tkinter import Menu, filedialog
from matplotlib.figure import Figure
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
import cv2
import os
from PIL import Image, ImageTk
from collections import defaultdict
import numpy as np
from ultralytics import YOLO

class VideoPlayer(tk.Tk):
    def __init__(self, video_path):
        super().__init__()
        self.title("Video Player GUI")
        self.geometry("1000x700")
        self.model = YOLO("med_yolo11n.pt")
        self.paused = False
        self.zoomed_frame = None
        self.prev_checkbox_status = {}
        self.track_history = defaultdict(list)
        self.cap = cv2.VideoCapture(video_path)

        self.create_menu()
        self.create_main_frames()
        self.create_left_panels()
        self.create_right_panels()
        self.create_bar_graph()
        self.create_controls()
        self.create_speed_slider()
        self.update_video()

    def create_menu(self):
        menu_bar = Menu(self)
        self.config(menu=menu_bar)
        file_menu = Menu(menu_bar, tearoff=0)
        file_menu.add_command(label="Open", command=self.open_file)
        file_menu.add_command(label="Save", command=self.save_file)
        file_menu.add_separator()
        file_menu.add_command(label="Exit", command=self.quit)
        menu_bar.add_cascade(label="File", menu=file_menu)
        edit_menu = Menu(menu_bar, tearoff=0)
        edit_menu.add_command(label="Undo", command=self.undo_action)
        edit_menu.add_command(label="Redo", command=self.redo_action)
        menu_bar.add_cascade(label="Edit", menu=edit_menu)
        help_menu = Menu(menu_bar, tearoff=0)
        help_menu.add_command(label="About", command=self.show_about)
        menu_bar.add_cascade(label="Help", menu=help_menu)

    def create_main_frames(self):
        self.left_frame = tk.Frame(self, width=300, height=700, padx=10, pady=10)
        self.left_frame.pack_propagate(False)
        self.left_frame.pack(side=tk.LEFT, fill=tk.Y, padx=5, pady=5)
        self.right_frame = tk.Frame(self, width=680, height=700, padx=10, pady=10)
        self.right_frame.pack_propagate(False)
        self.right_frame.pack(side=tk.RIGHT, fill=tk.BOTH, expand=True, padx=5, pady=5)

    def create_left_panels(self):
        self.top_left_panel = self.create_label_frame(self.left_frame, "Zoomed and Cropped Video")
        self.middle_left_panel = self.create_label_frame(self.left_frame, "Counted Types")
        self.bottom_left_panel = self.create_label_frame(self.left_frame, "Show Types")
        self.zoomed_label = tk.Label(self.top_left_panel)
        self.zoomed_label.pack(fill=tk.BOTH, expand=True)
        self.check_vars = [tk.IntVar() for _ in range(7)]
        self.clses = ["T1-Amorf_Bas", "T2-Akrozomal_Vakuol", "T3-Normal", "T4-Amorf_Bas-Kisa_Kuyruk", "T5-Amorf_Bas-Kisa_Kut_Kuyruk", "T6-Nuklear_Anomali", "T7-Uzun_Bas-Buyuk_Bas"]
        self.check_buttons = [self.create_check_button(self.bottom_left_panel, text, var) for text, var in zip(self.clses, self.check_vars)]

    def create_right_panels(self):
        self.video_panel = self.create_label_frame(self.right_frame, "Video", 680, 500)
        self.video_label = tk.Label(self.video_panel)
        self.video_label.pack(fill=tk.BOTH, expand=True)
        self.explanation_panel = self.create_label_frame(self.right_frame, "Explanation Area", 680, 100)
        self.explanation_label = tk.Label(self.explanation_panel, text="Explanation Area", wraplength=600)
        self.explanation_label.pack(fill=tk.X)

    def create_bar_graph(self):
        self.fig = Figure(figsize=(3, 2))
        self.ax = self.fig.add_subplot(111)
        self.canvas = FigureCanvasTkAgg(self.fig, master=self.middle_left_panel)
        self.canvas.draw()
        self.canvas.get_tk_widget().pack(fill=tk.BOTH, expand=True)
        self.update_bar_graph([5, 7, 3, 8, 4, 0, 0])

    def create_controls(self):
        self.play_button = tk.Button(self.right_frame, text="Play", command=self.play_video)
        self.play_button.pack(side=tk.LEFT, padx=5, pady=5)
        self.pause_button = tk.Button(self.right_frame, text="Pause", command=self.pause_video)
        self.pause_button.pack(side=tk.LEFT, padx=5, pady=5)
        self.save_button = tk.Button(self.right_frame, text="Save Frame", command=self.save_frame)
        self.save_button.pack(side=tk.LEFT, padx=5, pady=5)
        self.fullscreen_button = tk.Button(self.right_frame, text="Fullscreen", command=self.toggle_fullscreen)
        self.fullscreen_button.pack(side=tk.LEFT, padx=5, pady=5)

    def create_speed_slider(self):
        self.speed_slider = tk.Scale(self.right_frame, from_=1, to=100, orient=tk.HORIZONTAL, label="Speed")
        self.speed_slider.set(30)
        self.speed_slider.pack(side=tk.LEFT, padx=5, pady=5)

    def create_label_frame(self, parent, text, width=300, height=200):
        frame = tk.LabelFrame(parent, text=text, width=width, height=height)
        frame.pack_propagate(False)
        frame.pack(fill=tk.BOTH, expand=True, padx=5, pady=5)
        return frame

    def create_check_button(self, parent, text, var):
        chk = tk.Checkbutton(parent, text=text, variable=var, command=self.update_explanation_area)
        chk.pack(anchor=tk.W)
        return chk

    def open_file(self):
        filetypes = (("Video files", "*.mp4 *.avi *.mkv"), ("All files", "*.*"))
        filepath = filedialog.askopenfilename(title="Open Video File", initialdir=os.getcwd(), filetypes=filetypes)
        if filepath:
            self.change_video(filepath)

    def save_file(self):
        pass

    def undo_action(self):
        pass

    def redo_action(self):
        pass

    def show_about(self):
        pass

    def change_video(self, video_path):
        if self.cap.isOpened():
            self.cap.release()
        self.track_history.clear()
        self.cap = cv2.VideoCapture(video_path)

    def play_video(self):
        self.paused = False
        self.update_video()

    def pause_video(self):
        self.paused = True
        if hasattr(self, 'after_id'):
            self.after_cancel(self.after_id)

    def save_frame(self):
        ret, frame = self.cap.read()
        if ret:
            cv2.imwrite("saved_frame.png", frame)
            print("Frame saved as saved_frame.png")

    def toggle_fullscreen(self):
        self.attributes("-fullscreen", not self.attributes("-fullscreen"))

    def update_video(self):
        if self.paused:
            return
        ret, frame = self.cap.read()
        if ret:
            annotated_frame, best_box = self.detect_objects(frame)
            self.display_main_video(annotated_frame)  # Display the annotated frame with detections
            if best_box is not None:
                self.update_zoomed_frame(frame, best_box)  # Update the zoomed frame only when a detection occurs
            self.update_periodic_tasks()
        else:
            print("Warning: Failed to read frame from video")
        self.after_id = self.after(self.speed_slider.get(), self.update_video)

    def process_frame(self, frame):
        frame_with_rectangle = frame.copy()
        x, y = 150, 150
        result, best_box = self.detect_objects(frame_with_rectangle)
        if result is not None:
            frame_with_rectangle = result
        self.draw_rectangle(frame_with_rectangle, best_box, x, y)
        return frame_with_rectangle

    def draw_rectangle(self, frame, best_box, x, y):
        rect_width, rect_height = 100, 100
        if best_box is not None:
            x, y, w, h = map(int, best_box)
        cv2.rectangle(frame, (x - rect_width // 2, y - rect_height // 2), (x + rect_width // 2, y + rect_height // 2), (0, 255, 0), 2)
        self.update_explanation(f"Rectangle Coordinates: ({x - rect_width // 2}, {y - rect_height // 2}), ({x + rect_width // 2}, {y + rect_height // 2})")

    def detect_objects(self, frame):
        clss = self.get_checked_indices() or [2]
        results = self.model.track(frame, imgsz=(640, 480), conf=0.4, iou=0.5, tracker="bytetrack.yaml", persist=True, classes=clss)
        annotated_frame, best_box = self.extract_best_box(results)
        return annotated_frame, best_box

    def extract_best_box(self, results):
        best_box = None
        annotated_frame = None
        if results[0].boxes.id is not None:
            boxes, scores, classes = self.extract_boxes_scores_classes(results)
            best_box = self.find_most_probable_normal(boxes, scores, classes)
            track_ids = results[0].boxes.id.int().cpu().tolist()
            annotated_frame = self.annotate_frame(results, boxes, track_ids)
        return annotated_frame, best_box

    def extract_boxes_scores_classes(self, results):
        boxes = results[0].boxes.xywh.cpu().numpy()
        scores = results[0].boxes.conf.cpu().numpy()
        classes = results[0].boxes.cls.cpu().numpy()
        return boxes, scores, classes

    def find_most_probable_normal(self, boxes, scores, classes, normal_class_index=3):
        max_score = 0
        best_box = None
        for box, score, cls in zip(boxes, scores, classes):
            if cls == normal_class_index and score > max_score:
                max_score = score
                best_box = box
        return best_box

    def annotate_frame(self, results, boxes, track_ids):
        annotated_frame = results[0].plot(font_size=6)
        for box, track_id in zip(boxes, track_ids):
            x, y, w, h = box
            track = self.track_history[track_id]
            track.append((float(x), float(y)))
            if len(track) > 30:
                track.pop(0)
            points = np.hstack(track).astype(np.int32).reshape((-1, 1, 2))
            cv2.polylines(annotated_frame, [points], isClosed=False, color=(230, 230, 230), thickness=2)
        return annotated_frame

    def update_zoomed_frame(self, frame, best_box=None):
        height, width, _ = frame.shape
        cx, cy = self.get_zoom_center(best_box, width, height)
        x1, y1, x2, y2 = self.get_zoom_coordinates(cx, cy, width, height)
        
        # Debugging information
        print(f"Zoomed Frame Coordinates: ({x1}, {y1}), ({x2}, {y2})")
        
        self.zoomed_frame = frame[y1:y2, x1:x2].copy()
        
        # Check if the zoomed frame is empty
        if self.zoomed_frame.size == 0:
            print("Zoomed frame is empty!")
        else:
            print("Zoomed frame is not empty.")
        
        self.display_zoomed_video()  # Ensure the zoomed frame is displayed immediately

    def get_zoom_center(self, best_box, width, height):
        if best_box is not None:
            x, y, w, h = map(int, best_box)
            return x, y
        # Default center coordinates
        return width // 2, height // 2

    def get_zoom_coordinates(self, cx, cy, width, height, zoom_size=100):
        x1, y1 = max(0, cx - zoom_size // 2), max(0, cy - zoom_size // 2)
        x2, y2 = min(width, cx + zoom_size // 2), min(height, cy + zoom_size // 2)
        return x1, y1, x2, y2

    def display_zoomed_video(self):
        if self.zoomed_frame is not None:
            zoomed_frame_resized = cv2.resize(self.zoomed_frame, (self.zoomed_label.winfo_width(), self.zoomed_label.winfo_height()), interpolation=cv2.INTER_LINEAR)
            zoomed_frame_rgb = cv2.cvtColor(zoomed_frame_resized, cv2.COLOR_BGR2RGB)
            img = Image.fromarray(zoomed_frame_rgb)
            self.zoomed_imgtk = ImageTk.PhotoImage(image=img)  # Keep a reference to avoid garbage collection
            self.zoomed_label.imgtk = self.zoomed_imgtk
            self.zoomed_label.configure(image=self.zoomed_imgtk)

    def display_main_video(self, frame):
        if frame is not None and frame.size > 0:
            frame_resized = cv2.resize(frame, (self.video_label.winfo_width(), self.video_label.winfo_height()))
            frame_rgb = cv2.cvtColor(frame_resized, cv2.COLOR_BGR2RGB)
            img = Image.fromarray(frame_rgb)
            self.main_imgtk = ImageTk.PhotoImage(image=img)  # Keep a reference to avoid garbage collection
            self.video_label.imgtk = self.main_imgtk
            self.video_label.configure(image=self.main_imgtk)
        else:
            print("Warning: Empty frame passed to display_main_video")

    def update_explanation_area(self):
        current_checkbox_status = self.get_checkboxes_status()
        if current_checkbox_status != self.prev_checkbox_status:
            explanation_text = "Checkboxes Status:\n" + "\n".join([f"{k}: {'Checked' if v else 'Unchecked'}" for k, v in current_checkbox_status.items()])
            self.update_explanation(explanation_text)
            self.prev_checkbox_status = current_checkbox_status

    def update_periodic_tasks(self):
        if int(self.cap.get(cv2.CAP_PROP_POS_FRAMES)) % int(self.cap.get(cv2.CAP_PROP_FPS)) == 0:
            self.update_bar_graph([6, 8, 4, 9, 5, 0, 0])
        self.update_explanation_area()

    def update_bar_graph(self, data):
        labels = ['T1', 'T2', 'T3', 'T4', 'T5', 'T6', 'T7']
        self.ax.clear()
        self.ax.bar(range(len(data)), data)
        self.ax.set_xticks(range(len(data)))
        self.ax.set_xticklabels(labels)
        self.canvas.draw()

    def update_explanation(self, text):
        self.explanation_label.config(text=text)

    def get_checkboxes_status(self):
        return {chk.cget("text"): var.get() for chk, var in zip(self.check_buttons, self.check_vars)}

    def get_checked_indices(self):
        return [i for i, var in enumerate(self.check_vars) if var.get() == 1]


In [ ]:

if __name__ == "__main__":
    video_path = "/Users/sso/Programming/codes/python/projects/103_med/videos/video1.mp4"
    app = VideoPlayer(video_path)
    app.mainloop()


0: 384x480 (no detections), 26.3ms
Speed: 1.4ms preprocess, 26.3ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 480)

0: 384x480 (no detections), 16.4ms
Speed: 1.2ms preprocess, 16.4ms inference, 0.2ms postprocess per image at shape (1, 3, 384, 480)

0: 384x480 (no detections), 16.3ms
Speed: 1.2ms preprocess, 16.3ms inference, 0.2ms postprocess per image at shape (1, 3, 384, 480)

0: 384x480 (no detections), 15.7ms
Speed: 0.8ms preprocess, 15.7ms inference, 0.2ms postprocess per image at shape (1, 3, 384, 480)

0: 384x480 (no detections), 14.9ms
Speed: 0.9ms preprocess, 14.9ms inference, 0.2ms postprocess per image at shape (1, 3, 384, 480)

0: 384x480 (no detections), 15.2ms
Speed: 1.0ms preprocess, 15.2ms inference, 0.2ms postprocess per image at shape (1, 3, 384, 480)

0: 384x480 (no detections), 22.7ms
Speed: 1.2ms preprocess, 22.7ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 480)

0: 384x480 (no detections), 16.1ms
Speed: 0.9ms preprocess, 16.1ms in